## Assignment 1: MongoDB - Ronald Garcia 

This assignment is based on content discussed in Module 2: Introduction to MongoDB

## Learning outcomes 

The purpose of this assignment is for learners to be able to:
- Familarize with JSON document syntax
- Understand basic MongoDB CRUD operations
- Understand MongoDB data pipelines to run aggregate queries

In this assignment, you will make use of the sample data provided in Module 2.  

This dataset has 3 collections: Employee, Workplace and Address.  You will import this data into your local MongoDB database.

Required imports for this project are given below. Make sure you have all libraries required for this project installed. You may use conda or pip based on your set up.

## Setup Notes

**Please note** These instructions are duplicated here for your reference. You would have encountered them originally in Module 2 Part 2. It is expected that you have already taken the steps to set up and run MongoDB.

We will be using MongoDB Community Edition.  The MongoDB database <b>MUST</b> be installed and running locally before continuing with this notebook.  We will need to install two packages using the Anaconda package manager:

1. [Mongodb](https://www.mongodb.com/) - this package contains the mongodb database 
2. [PyMongo](http://api.mongodb.com/python/current/) - this packages contains the python driver that will allow us to communicate with the mongodb database.

#### Install

1. Open a command line terminal and execute the following commad to install mongodb.
```console
conda install -c anaconda mongodb
```
2. Open a command line terminal and execute the following commad to install pymongo packge.
```console
conda install -c anaconda pymongo
```

#### Run Mongodb in Windows
1. MongoDB requires a data directory to store all data. MongoDB’s default data directory path is \data\db. Create this folder using the following commands from a Command Prompt:
```console
md \data\db
```

2. To start MongoDB, run mongod.exe. For example, from the Command Prompt:
```console
"C:\Program Files\MongoDB\Server\3.2\bin\mongod.exe"
```

#### Run Mongodb in Mac
1. MongoDB requires a data directory to store all data. MongoDB’s default data directory path is /data/db. Create this folder using the following commands from a Command Prompt.  Note that we run the command as a super user using the "sudo" command:
```console
sudo mkdir /data/db/
```

2. To start MongoDB, run mongod.exe. For example, from the Command Promp.  Note that we run the command as a super user using the "sudo" command:
```console
sudo mongod --config /usr/local/etc/mongod.conf
```

In [1]:
#required imports
import os
import json
import datetime
import pymongo
import pprint
import pandas as pd
import numpy as np
from pymongo import MongoClient
print('Mongo version', pymongo.__version__)

Mongo version 3.11.0


We first need to connect to our locally running MongoDB database (<b>Make sure your database is running on your machine</b>). We will use the MongoClient to connect to a local 'test' database that is running on port 27017 (this is the default port).

In [2]:
client = MongoClient('localhost', 27017)
db = client.assignment1

After installing necessary modules proceed to import the data into your database.

In [3]:
# Let's delete any existing collections in our database
db.workplace.drop()
db.address.drop()
db.employee.drop()

# Import our files into our three collections
with open('Data/Employee.json') as f:
    db.employee.insert_many(json.load(f))
with open('Data/Workplace.json') as f:
    db.workplace.insert_many(json.load(f))
with open('Data/Address.json') as f:
    db.address.insert_many(json.load(f))

#### Question 1 (10 Marks)

The address collection contains employee from different ages and interests.  Perform a simple query to list all employees that are less than or equal to 50 and like Cooking.

__NOTE:__ the following shows the structure of an Employee document that will help you construct the query.

In [4]:
pprint.pprint(client.assignment1.employee.find_one())

{'_id': '9f39da36-82cc-4353-ab90-d616105fa7c1',
 'address_id': 'b6c0b50a-d0e3-43bf-a2a4-8d4674c2a7e8',
 'age': 40,
 'email': 'ih@ri.ro',
 'firstname': 'Emilie',
 'interests': ['Bowling', 'Cooking', 'Golf', 'Swimming'],
 'lastname': 'Woods',
 'workplace_id': 'a32bf18d-e0e5-48f2-a851-aa49c80f9460'}


In [5]:
cursor_q1 = client.assignment1.employee.find({
    'age': {'$lte': 50 }, 
    'interests': { '$in': [ 'Cooking' ] }
    }).sort("age", pymongo.DESCENDING)
pd.DataFrame(list(cursor_q1))

,_id,firstname,lastname,age,email,interests,address_id,workplace_id
0,66894dba-4ff0-4545-b7cc-eb6a5bd551c5,Clara,Butler,49,do@womjip.by,"[Soccer, Cooking, Cycling]",4dcebe6e-0787-4158-9744-3d721098cec4,50275ad1-8140-4e79-8818-21793e3eb0a3
1,b2a2ae86-f4f6-4da5-ade6-ee1cd135daf3,Gavin,Conner,49,ah@jopah.uz,"[Rubgy, Cooking, Bowling]",ef5a17a2-6f11-49b8-a0fd-e0bddc56f185,50275ad1-8140-4e79-8818-21793e3eb0a3
2,6157dc3b-ee2d-463a-b62b-1cd4eed7d575,Dominic,Wade,48,co@pog.nz,"[Cycling, Dancing, Cooking]",8d162eb9-eefc-4b7e-bcad-6fb7a02b15b0,cb795afb-8dc3-482f-b3a0-14229a280afa
3,9f39da36-82cc-4353-ab90-d616105fa7c1,Emilie,Woods,40,ih@ri.ro,"[Bowling, Cooking, Golf, Swimming]",b6c0b50a-d0e3-43bf-a2a4-8d4674c2a7e8,a32bf18d-e0e5-48f2-a851-aa49c80f9460
4,4bc070ca-f849-4eeb-8ab2-98fe3c0861c0,Francis,Harris,38,pa@sodej.ck,"[Dancing, Swimming, Cooking, Bowling, Cycling]",fc607f68-81c8-4ca8-8a9e-30239ccae40f,2b87eb84-e5b4-4f2c-9e13-dc3ba20a7f7f
5,2bfc0479-cb15-46e1-94fa-801404791b21,Peter,Jimenez,37,usope@tega.bh,"[Cooking, Swimming, Cricket]",77ce0c50-afd6-47e8-86d7-b1b372e8deb2,cb795afb-8dc3-482f-b3a0-14229a280afa
6,840184a3-4c4d-4b15-8813-30fca6e7827b,Delia,Douglas,36,me@wak.ne,"[Cricket, Cooking, Hiking, Dancing, Tennis]",456e18ae-c2f8-443e-899d-f2b893499695,b2a2844d-aeab-4602-b74c-01bf3b8e9c78
7,cc3e389d-be0d-467a-ba70-7c84f6504911,Myrtle,Little,36,saj@far.zm,"[Cooking, Cycling, Hiking, Rubgy, Bowling, Dan...",0d3a90d5-11bd-4f96-91ba-7e7ef80888f0,2b87eb84-e5b4-4f2c-9e13-dc3ba20a7f7f
8,457ef68c-9651-4925-bca0-15e246661d19,Alta,Sharp,34,jus@goal.bn,"[Cricket, Cycling, Rubgy, Golf, Cooking, Dancing]",5e97658e-5809-41b8-a088-eddbd81f86a7,b2a2844d-aeab-4602-b74c-01bf3b8e9c78
9,581f59a2-ff4c-407e-b58f-e4f292208928,Rena,Johnson,33,vasberet@his.nz,"[Cycling, Swimming, Soccer, Cooking]",1cf24906-c700-41bb-a4c8-bba3c3ae30d9,2b87eb84-e5b4-4f2c-9e13-dc3ba20a7f7f


#### Question 2  (10 Marks)

Insert a new Employee with the following properties:

* First Name: Jake 
* Last Name: Sample
* Email: jakesample@email.com
* Age: 26
* Interest: Biking, Hiking

Also, this employee works for 'Union Planters Corp' and lives at '573 Wojhas Square, Victoria'.
Verify that the insert succeeded and display the generated employees _id attribute.

__HINT__ An Employee document references a Workplace and Address document

In [6]:
#Let's find the workplace and address before inserting the new employee
cursor_q2_workplace = client.assignment1.workplace.find({
    'name': 'Union Planters Corp'
    })
pd.DataFrame(list(cursor_q2_workplace))

,_id,name,website,industry,address_id
0,5345fcb9-6297-4b9f-aa15-cbee8460f28f,Union Planters Corp,http://www.unionplanternscorp.com,Aerospace,9949fe3b-99ec-4485-b91d-823925db7d28


In [7]:
cursor_q2_address = client.assignment1.address.find({
    'address': '573 Wojhas Square'
    })
pd.DataFrame(list(cursor_q2_address))

,_id,address,postalcode,city,province
0,91b5b7b3-2309-4e8a-8247-cd66d626ef0c,573 Wojhas Square,A7D 5A3,Victoria,BC


In [8]:
#Now we can insert our new employee
client.assignment1.employee.insert_one({
    "firstname": "Jake",
    "lastname": "Sample",
    "age": 26,
    "email": "jakesample@email.com",
    "interests": ["Biking", "Hiking"],
    "address_id": "91b5b7b3-2309-4e8a-8247-cd66d626ef0c",
    "workplace_id": "5345fcb9-6297-4b9f-aa15-cbee8460f28f"
})

In [9]:
pprint.pprint(client.assignment1.employee.find_one({
 'firstname': 'Jake'
}))

{'_id': ObjectId('60275b53c84d8e5c6c496742'),
 'address_id': '91b5b7b3-2309-4e8a-8247-cd66d626ef0c',
 'age': 26,
 'email': 'jakesample@email.com',
 'firstname': 'Jake',
 'interests': ['Biking', 'Hiking'],
 'lastname': 'Sample',
 'workplace_id': '5345fcb9-6297-4b9f-aa15-cbee8460f28f'}


#### Question 3 (10 Marks)

Delete all employees that work for 'Great Plains Energy Inc' and are greater than 46 years old and likes 'Tennis'. Once you delete the employees verify the number of employees deleted.

In [10]:
#Let's find the ID for 'Great Plains Energy Inc' 
cursor_q3_workplace = client.assignment1.workplace.find({
    'name': 'Great Plains Energy Inc.'
    })
pd.DataFrame(list(cursor_q3_workplace))

,_id,name,website,industry,address_id
0,a32bf18d-e0e5-48f2-a851-aa49c80f9460,Great Plains Energy Inc.,http://www.greatplainsenergy.com,Agriculture,c35b4c7f-b7de-431d-bd60-4026490cd61c


In [11]:
#Let's verify our query criteria before deleting the employees
cursor_q3_delete = client.assignment1.employee.find({
    'age': {'$gt': 46 }, 
    'interests': { '$in': [ 'Tennis' ] },
    'workplace_id': 'a32bf18d-e0e5-48f2-a851-aa49c80f9460'
    }).sort("age", pymongo.DESCENDING)
pd.DataFrame(list(cursor_q3_delete))

,_id,firstname,lastname,age,email,interests,address_id,workplace_id
0,dfce1d98-15f6-4b75-86f6-cd5d05cfdc04,Sadie,Park,80,vukhik@wejoce.nf,"[Golf, Fishing, Bowling, Dancing, Cooking, Ten...",2987cea4-b71d-4338-b0ef-a9b5feb3a138,a32bf18d-e0e5-48f2-a851-aa49c80f9460
1,153ba346-b33d-4611-802c-fd95c7cb078c,Adelaide,Curry,71,ov@sutec.mr,[Tennis],8264af1e-1c55-4ed8-96f9-e3b9e1eff7d5,a32bf18d-e0e5-48f2-a851-aa49c80f9460
2,86417f71-18cd-41c5-ad01-24b31ff2074f,Lucy,May,65,bi@udtaofe.tj,[Tennis],fe80c95a-a0e9-42c9-aa6a-56f27020842d,a32bf18d-e0e5-48f2-a851-aa49c80f9460
3,8e082f05-13b7-485d-89c4-259f1103f95c,Lester,Summers,48,wi@uvafej.pt,"[Bowling, Tennis]",62996de2-fcda-4894-95d5-fef0d11d92f4,a32bf18d-e0e5-48f2-a851-aa49c80f9460


In [12]:
#Now we delete all the employees with the given criteria
delete_result = client.assignment1.employee.delete_many({
    'age': {'$gt': 46 }, 
    'interests': { '$in': [ 'Tennis' ] },
    'workplace_id': 'a32bf18d-e0e5-48f2-a851-aa49c80f9460'
    })
print("Delete Acknowledged:" + str(delete_result.acknowledged))
print("Delete Count:" + str(delete_result.deleted_count))

Delete Acknowledged:True
Delete Count:4


#### Question 4 (12 Marks)
Add a new field called 'industry' to all employees that work for 'Health Net Inc.'.

__HINT__ All a new field to a document is like updating the document

In [13]:
#Let's find the record for the 'Health Net Inc.' workplace
query_workplace = client.assignment1.workplace.find( { 'name': 'Health Net Inc.' } )
df_HealthNet = pd.DataFrame(query_workplace)
df_HealthNet

,_id,name,website,industry,address_id
0,a222385c-342c-43ea-adbc-9b487a2ee2be,Health Net Inc.,http://www.healthnetinc.com,Construction,1ed298fc-20ab-4750-ac38-fed1e60964af


In [14]:
#Now we are going to update all the Health Net employees 
cursor_q4_update = client.assignment1.employee.update_many(
     {'workplace_id': df_HealthNet.iloc[0]['_id']},
     {'$set': { 'industry' : 'New Industry' }}
    )
print("Update Acknowledged:" + str(cursor_q4_update.acknowledged))
print("Modified Count:" + str(cursor_q4_update.modified_count))

Update Acknowledged:True
Modified Count:14


In [15]:
#Let's see the updates
cursor_q4= client.assignment1.employee.find({
     'workplace_id': df_HealthNet.iloc[0]['_id']
    })
pd.DataFrame(list(cursor_q4))

,_id,firstname,lastname,age,email,interests,address_id,workplace_id,industry
0,ad4fd8bd-b732-4388-8469-5395b497df63,Marc,Castro,42,tasnimnag@tefifdum.bo,"[Golf, Cricket, Rubgy, Hiking, Fishing, Dancing]",91b32548-65b6-4fd9-91a1-2d86b04b8eea,a222385c-342c-43ea-adbc-9b487a2ee2be,New Industry
1,6222b4f5-6fd3-4c72-b058-710fb568b869,Lucinda,Gomez,46,ruupeke@oh.om,"[Golf, Rubgy, Tennis]",59516270-19f4-47e8-95ae-3e83fa5f230c,a222385c-342c-43ea-adbc-9b487a2ee2be,New Industry
2,ca189ae9-d04b-4ae0-9b4a-b725196c31d2,Emilie,Robertson,37,vom@calfim.se,[Hiking],16ff1778-b687-42ef-aace-36cedc05f521,a222385c-342c-43ea-adbc-9b487a2ee2be,New Industry
3,ed499274-f17f-4a4c-8327-7e740e6dd9c1,Josephine,Mills,36,newi@to.in,"[Tennis, Bowling, Cricket]",01185f9b-b00f-4f2d-9878-0c989fe00fbc,a222385c-342c-43ea-adbc-9b487a2ee2be,New Industry
4,6a3bd505-56e6-46ec-bef1-2eef1fd3643c,Vernon,Edwards,24,disbuivi@efu.id,[Soccer],a8f07070-cfb0-483e-a07a-b2adf9e3d4b0,a222385c-342c-43ea-adbc-9b487a2ee2be,New Industry
5,5b8bb404-5741-46a9-84f2-81248ffd2840,Glen,Turner,43,kazvasi@nojrek.aq,"[Golf, Hiking]",fc2b6b58-8392-4f3d-b1c8-07f4df87dc24,a222385c-342c-43ea-adbc-9b487a2ee2be,New Industry
6,c7cf8650-f391-4f2b-bbb5-60225729e998,Jeremiah,Payne,49,himosewej@repan.bw,"[Cricket, Fishing]",44cc24b9-1d60-45cf-be0d-a225f35d52ab,a222385c-342c-43ea-adbc-9b487a2ee2be,New Industry
7,9ebcf9fc-e193-4ab7-9c38-10c2d6bf6036,Alan,Berry,76,zut@zoksa.kw,"[Bowling, Cooking, Rubgy, Fishing, Golf, Cricket]",d34f9d8c-c1a0-4653-8a44-d2fe65714956,a222385c-342c-43ea-adbc-9b487a2ee2be,New Industry
8,fd677b4b-6eef-4777-bc3a-25d9a86134fa,Mattie,Fisher,37,pelohno@hotivgu.hn,"[Cycling, Fishing, Soccer, Rubgy, Hiking]",98fa15f0-5b73-4fb9-a5a4-1332e0f5b58b,a222385c-342c-43ea-adbc-9b487a2ee2be,New Industry
9,cb369c19-4bbc-449f-bff6-b198439d0d7b,Adele,Benson,43,oka@lilozfaz.gn,"[Swimming, Hiking, Tennis]",d5b7473d-c82b-4956-bce3-80497ce6c015,a222385c-342c-43ea-adbc-9b487a2ee2be,New Industry


#### Question 5 (10 Marks)

Create an aggregate query to count the number of employees for each company and sort the output from largest employee count to lowest employee count.

__NOTE__ you will use a pipeline to achieve the computed result.  You should produce a result similar to the following table (the following table contains fake data)
<table>
    <tr><th></th><th>_id</th><th>count</th></tr>
    <tr><td>0</td><td>[Equity Residential Properties Trust]</td><td>19</td></tr>
    <tr><td>...</td><td>...</td><td>...</td></tr>
    <tr><td>7</td><td>[Bell Microproducts Inc.]</td><td>6</td></tr>
    <tr><td>8</td><td>[Kemet Corp.]</td><td>1</td></tr>
</table>

__HINT__ you should make use of the \\$lookup, \\$group and \\$sort pipeline operations

In [16]:
#Let's create our pipeline
count_employees_pipeline = [
    {
        "$lookup":
        {
        "from": "workplace",
        "localField": "workplace_id",
        "foreignField": "_id",
        "as": "workplace"
        }
    },
    {
        "$group": {"_id": "$workplace.name", "count": {"$sum": 1}}
    },
    {
        "$sort": { "count": -1 }
    }
]
query_q5 = client.assignment1.employee.aggregate(count_employees_pipeline)
df_num_employees = pd.DataFrame((query_q5))
df_num_employees.rename(columns={'_id': 'Company', 'count': 'Number of employees'}, inplace=True)
df_num_employees

,Company,Number of employees
0,[Hilton Solutions],15
1,[Health Net Inc.],14
2,[Aetna Inc.],13
3,[Bell Microproducts Inc.],11
4,[Equity Office Properties Trust],10
5,[Union Planters Corp],10
6,[Equity Residential Properties Trust],7
7,[Xcel Bear Inc],6
8,[Kemet Corp.],6
9,[Great Plains Energy Inc.],5
